In [8]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import warnings
pd.options.display.max_columns = 100000
warnings.filterwarnings('ignore')

if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
else:
    print("MT5 successfully initialised.\n")

MT5 successfully initialised.



In [19]:
def download_stocks(symbols, n_dfPoint=2000, timeframe=mt5.TIMEFRAME_D1):
    dfs = []  # List to hold individual dataframes
    
    # Initialize MT5 connection
    if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
        return None

    for symbol in symbols:
        # Fetch the data using MT5 API
        data = mt5.copy_rates_from_pos(symbol, timeframe, 1, n_dfPoint)
        
        # Convert the data into a DataFrame
        df = pd.DataFrame(data)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)
        
        # Rename the 'close' column to the stock symbol and append to the dfs list
        dfs.append(df[['close']].rename(columns={'close': symbol}))

    # Cleanup the MT5 connection
    mt5.shutdown()

    # Concatenate the individual dataframes into a single dataframe
    final_df = pd.concat(dfs, axis=1)
    return final_df

# Example usage:
symbols = ['AXREIT','TIMECOM','ALLIANZ','9199.KL','7218.KL',
           '7293.KL','6017.KL','5131.KL','5162.KL','3069.KL',
           '7090.KL','5138.KL','9598.KL']
df = download_stocks(symbols)
print(df)


KeyError: 'time'

In [18]:
timeframe = mt5.TIMEFRAME_D1
symbol = 'TOPGLOV'
ref_symbol = None
n_dfPoint=2000
df = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, 1, n_dfPoint))
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)
df

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2017-01-23,5.160,5.320,5.160,5.230,865,0,3037900
2017-01-24,5.230,5.270,5.190,5.230,1664,0,1433400
2017-01-25,5.250,5.250,5.180,5.220,1266,0,2770400
2017-01-26,5.200,5.230,5.190,5.200,704,0,2798500
2017-01-27,5.180,5.200,5.130,5.200,666,0,1267100
...,...,...,...,...,...,...,...
2023-10-26,0.720,0.730,0.715,0.725,1934,5,21561845
2023-10-27,0.725,0.730,0.715,0.720,1540,5,17598151
2023-10-30,0.720,0.720,0.710,0.715,1137,5,10108390
